In [1]:
import ankh
import numpy as np
import pandas as pd
import torch
import yaml
import logging
from torch.optim import AdamW
from torch.utils.data import DataLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau

from data_prepare import make_folds, prepare_embed_df
from torch_utils import (
    SequenceDataset,
    custom_collate_fn,
    CustomBatchSampler,
    train_fn,
    validate_fn,
    evaluate_fn
)

In [2]:
import clearml
from clearml import Logger, Task

In [3]:
clearml.browser_login()
task = Task.init(
    project_name="DBPs_search",
    task_name="Finetune Ankh v4",
    output_uri=True,
)
logger = Logger.current_logger()

ClearML Task: created new task id=9b912b2281484adca0cba97e26b1a38d
2024-06-20 14:05:45,131 - clearml.Task - INFO - Storing jupyter notebook directly as code
ClearML results page: https://app.clear.ml/projects/45ba7ff7a93646a8a76d1950065cf1d5/experiments/9b912b2281484adca0cba97e26b1a38d/output/log
ClearML Monitor: Could not detect iteration reporting, falling back to iterations as seconds-from-start


In [2]:
with open("config.yml", "r") as f:
    config = yaml.safe_load(f)

In [20]:
task.connect_configuration(config)

NameError: name 'task' is not defined

In [3]:
input_dim = config["model_config"]["input_dim"]
nhead = config["model_config"]["nhead"]
hidden_dim = config["model_config"]["hidden_dim"]
num_hidden_layers = config["model_config"]["num_hidden_layers"]
num_layers = config["model_config"]["num_layers"]
kernel_size = config["model_config"]["kernel_size"]
dropout = config["model_config"]["dropout"]
pooling = config["model_config"]["pooling"]


epochs = config["training_config"]["epochs"]
lr = config["training_config"]["lr"]
factor = config["training_config"]["factor"]
patience = config["training_config"]["patience"]
min_lr = config["training_config"]["min_lr"]
batch_size = config["training_config"]["batch_size"]
seed = config["training_config"]["seed"]
num_workers = config["training_config"]["num_workers"]

In [4]:
DEVICE = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")

In [5]:
def set_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)


set_seed(seed)

train - pd dataframe

In [57]:
# binary_classification_model = ankh.ConvBertForBinaryClassification(
#     input_dim=1536,
#     nhead=7,
#     hidden_dim=1723,
#     num_hidden_layers=2,
#     num_layers=1,
#     kernel_size=7,
#     dropout=dropout,
#     pooling=pooling,
# )

In [58]:
# binary_classification_model = binary_classification_model.to(DEVICE)

In [51]:
# a, b = next(iter(train_dataloader))

In [59]:
# output = binary_classification_model(a.to(DEVICE), b.to(DEVICE).unsqueeze(1))

In [33]:
# optimizer = AdamW(binary_classification_model.parameters(), lr=float(lr))
# scheduler = ReduceLROnPlateau(
#     optimizer, mode="min", factor=factor, patience=patience, min_lr=float(min_lr)
# )

In [7]:
df = prepare_embed_df(csv_path="../data/ready_data/train_pdb186.csv")
train_folds, valid_folds = make_folds(df)

In [8]:
df.label.value_counts()

1    34325
0    34325
Name: label, dtype: int64

In [36]:
for i in range(len(train_folds)):
    train_dataset = SequenceDataset(train_folds[i])
    train_sampler = CustomBatchSampler(train_dataset, batch_size)
    train_dataloader = DataLoader(
        train_dataset,
        num_workers=num_workers,
        batch_sampler=train_sampler,
        collate_fn=custom_collate_fn,
    )

    valid_dataset = SequenceDataset(valid_folds[i])
    valid_sampler = CustomBatchSampler(valid_dataset, batch_size)
    valid_dataloader = DataLoader(
        valid_dataset,
        num_workers=num_workers,
        batch_sampler=valid_sampler,
        collate_fn=custom_collate_fn,
    )

    binary_classification_model = ankh.ConvBertForBinaryClassification(
        input_dim=input_dim,
        nhead=nhead,
        hidden_dim=hidden_dim,
        num_hidden_layers=num_hidden_layers,
        num_layers=num_layers,
        kernel_size=kernel_size,
        dropout=dropout,
        pooling=pooling,
    )

    binary_classification_model = binary_classification_model.to(DEVICE)
    optimizer = AdamW(binary_classification_model.parameters(), lr=float(lr))
    scheduler = ReduceLROnPlateau(
        optimizer, mode="min", factor=factor, patience=patience, min_lr=float(min_lr)
    )

    best_val_loss = float("inf")
    best_model_path = f"checkpoints/pdb20000_best_model_{i}.pth"

    for epoch in range(epochs):
        train_loss = train_fn(binary_classification_model, train_dataloader, optimizer, DEVICE)
        valid_loss, metrics = validate_fn(
            binary_classification_model, valid_dataloader, scheduler, DEVICE
        )

        logger.report_scalar(
            title=f"Loss model {i}",
            series="train loss",
            value=train_loss,
            iteration=epoch,
        )
        logger.report_scalar(
            title=f"Loss model {i}",
            series="valid loss",
            value=valid_loss,
            iteration=epoch,
        )

        for metric_name, metric_value in metrics.items():
            logger.report_scalar(
                title=f"Metrics model {i}",
                series=metric_name,
                value=metric_value,
                iteration=epoch,
            )

        if valid_loss < best_val_loss:
            best_val_loss = valid_loss
            torch.save(binary_classification_model.state_dict(), best_model_path)
            message = f"Saved Best Model on epoch {epoch} with Validation Loss: {best_val_loss}"
            logger.report_text(message, level=logging.DEBUG, print_console=False)

2024-06-20 14:15:22,329 - clearml.storage - ERROR - Exception encountered while uploading Failed uploading object /DBPs_search/Finetune Ankh v3.9b912b2281484adca0cba97e26b1a38d/models/best_model_0.pth (413): <!doctype html>
<html lang=en>
<title>413 Request Entity Too Large</title>
<h1>Request Entity Too Large</h1>
<p>The data value transmitted exceeds the capacity limit.</p>

2024-06-20 14:15:22,336 - clearml.Task - INFO - Failed model upload
2024-06-20 14:19:39,146 - clearml.storage - ERROR - Exception encountered while uploading Failed uploading object /DBPs_search/Finetune Ankh v3.9b912b2281484adca0cba97e26b1a38d/models/best_model_0.pth (413): <!doctype html>
<html lang=en>
<title>413 Request Entity Too Large</title>
<h1>Request Entity Too Large</h1>
<p>The data value transmitted exceeds the capacity limit.</p>

2024-06-20 14:19:39,154 - clearml.Task - INFO - Failed model upload
2024-06-20 14:27:31,679 - clearml.storage - ERROR - Exception encountered while uploading Failed uploadi

Inference average best models

In [52]:
models = []
for i in range(5):
    binary_classification_model = ankh.ConvBertForBinaryClassification(
        input_dim=input_dim,
        nhead=nhead,
        hidden_dim=hidden_dim,
        num_hidden_layers=num_hidden_layers,
        num_layers=num_layers,
        kernel_size=kernel_size,
        dropout=dropout,
        pooling=pooling,
    )

    path_model = f"checkpoints/pdb2272_best_model_{i}.pth"
    binary_classification_model.load_state_dict(torch.load(path_model))
    binary_classification_model.eval()  # Set the model to evaluation mode
    models.append(binary_classification_model)

Testing on benchmark pdb2272

In [9]:
test_df = prepare_embed_df(
    embedding_path="../../../../ssd2/dbp_finder/ankh_embeddings/pdb186_2d.h5",
    csv_path="../data/embeddings/input_csv/pdb186.csv",
)

In [10]:
test_df.label.value_counts()

1    93
0    93
Name: label, dtype: int64

In [11]:
test_df

,identifier,sequence,label,embedding
0,2L8DA,GAMGMPNRKYADGEVVMGRWPGSVLYYEVQVTSYDDASHLYTVKYK...,1,"[[0.016087042, -0.005283513, 0.008722133, 0.00..."
1,2LI6A,QSLNPALQEKISTELNNKQYELFMKSLIENCKKRNMPLQSIPEIGN...,1,"[[0.023384146, 0.009197159, -0.023114325, -0.0..."
2,2LJ6A,MPSSKPLAEYARKRDFRQTPEPSGRKPRKDSTGLLRYCVQKHDASR...,1,"[[0.014131906, 0.0005972403, -0.018032532, -0...."
3,2LUYA,HMGKNDNDALIMCMRCRKVKGIDSYSKTQWSKTFTFVRGRTVSVSD...,1,"[[0.028977185, -0.0040170737, -0.0045093386, 0..."
4,2LVSA,MPSVNDSLDIVEKLYKDGVPVKEIAKRSNNSMSTVYKALEKLEAMG...,1,"[[0.01739704, -0.003137477, 0.007429028, -0.00..."
...,...,...,...,...
181,4HWXA,GSAHGPSAMVFTVIQGSGEPTDTVLRATTLSCAYTAEGTHPAPRAA...,0,"[[0.04853192, -0.008373539, 0.0075779436, 0.01..."
182,4HY3A,MHHHHHHSSGVDLGTENLYFQSMTNTERPLAISAPEPRSLDLIFSD...,0,"[[-0.019954232, -0.033106994, 0.02529409, 0.00..."
183,4I1DA,SNAQITFVSQGGAYQAAQTVAILDPSAKKLGITINQDSIPDAWPAI...,0,"[[0.0064985356, -0.017093375, -0.009304281, 0...."
184,4I8DA,AVVPPAGTPWGTAYDKAKAALAKLNLQDKVGIVSGVGWNGGPCVGN...,0,"[[0.011188501, -0.00968882, -0.022511976, -0.0..."


: 

In [59]:
testing_set = SequenceDataset(test_df)
testing_dataloader = DataLoader(
    testing_set,
    num_workers=num_workers,
    shuffle=False,
    batch_size=1,
)

In [67]:
metrics = evaluate_fn(models, testing_dataloader, DEVICE)

In [70]:
metrics_df = pd.DataFrame(metrics, index=["pdb20000"])

In [71]:
logger.report_table(title="pdb20000", series="Metrics", table_plot=metrics_df)

In [72]:
task.close()